In [2]:
readlines("input")

5-element Vector{String}:
 "#############"
 "#...........#"
 "###D#A#B#C###"
 "  #B#A#D#C#"
 "  #########"

Try manual solution

In [3]:
9000 + 30 + 5 + 5 + 50 + 40 + 5000 + 500 + 500 + 3000 + 3000

21130

Not the right answer 😥

In [4]:
300 + 300 + 9000 + 30 + 5 + 5 + 50 + 40 + 6000 + 500 + 500

16730

Still not the right answer 😥

Okay then, let's model it...

In [11]:
target_x = Dict('A'=>3, 'B'=>5, 'C'=>7, 'D'=>9)

Dict{Char, Int64} with 4 entries:
  'C' => 7
  'D' => 9
  'A' => 3
  'B' => 5

In [77]:
step_cost = Dict('A'=>1, 'B'=>10, 'C'=>100, 'D'=>1000)

Dict{Char, Int64} with 4 entries:
  'C' => 100
  'D' => 1000
  'A' => 1
  'B' => 10

In [84]:
empty_spaces = Set([(i,1) for i in [1,2,4,6,8,10,11]])
positions = [(3,2,'D'),(3,3,'B'),(5,2,'A'),(5,3,'A'),(7,2,'B'),(7,3,'D'),(9,2,'C'),(9,3,'C')]
possibilities = [(positions, empty_spaces, 0)]

1-element Vector{Tuple{Vector{Tuple{Int64, Int64, Char}}, Set{Tuple{Int64, Int64}}, Int64}}:
 ([(3, 2, 'D'), (3, 3, 'B'), (5, 2, 'A'), (5, 3, 'A'), (7, 2, 'B'), (7, 3, 'D'), (9, 2, 'C'), (9, 3, 'C')], Set([(8, 1), (1, 1), (4, 1), (6, 1), (2, 1), (10, 1), (11, 1)]), 0)

In [28]:
all_spaces = Set([empty_spaces..., [(x[1],x[2]) for x in positions]...])
occupied(empty) = setdiff(all_spaces, empty)

occupied (generic function with 1 method)

In [29]:
occupied(empty_spaces)

Set{Tuple{Int64, Int64}} with 8 elements:
  (9, 2)
  (9, 3)
  (3, 2)
  (5, 2)
  (3, 3)
  (5, 3)
  (7, 2)
  (7, 3)

In [15]:
function is_finished(possibility)
    (pos, space, cost) = possibility
    for p in pos
        if p[2]<2 || p[1] != target_x[p[3]]
            return false
        end
    end
    return true
end

is_finished (generic function with 1 method)

In [17]:
is_finished(possibilities[1])

false

In [60]:
function way_blocked(from, to, space)
    occ = occupied(space)
    if from[2]==3 && (from[1],2) ∈ occ
        return true
    end
    if to[2]==3 && (to[1],2) ∈ occ
        return true
    end
    return length(filter(x->x[1] ∈ min(from[1],to[1])+1:max(from[1],to[1])-1 && x[2]==1,occ)) > 0
end

way_blocked (generic function with 1 method)

In [51]:
positions[1],empty_spaces[1],empty_spaces

((3, 2, 'D'), (1, 1), [(1, 1), (2, 1), (4, 1), (6, 1), (8, 1), (10, 1), (11, 1)])

In [52]:
way_blocked(positions[1],empty_spaces[1],empty_spaces)

false

In [53]:
way_blocked(positions[1],positions[3],empty_spaces)

true

In [97]:
function move_possible(possibility, from, to)
    (pos, space, cost) = possibility
    if from[2]==1 && to[2]==1
        return false
    end
    if to[2]==3
        if to[1] != target_x[from[3]] || way_blocked(from, to, space)
            return false
        end
    elseif to[2]==2
        if from[1]==to[1] || to[1] != target_x[from[3]] || (to[1],3,from[3]) ∉ pos || way_blocked(from, to, space)
            return false
        end
    else
        if way_blocked(from, to, space) || (from[1] == target_x[from[3]] && (from[1],3,from[3]) ∈ pos)
            return false
        end
    end
    return true
end

move_possible (generic function with 1 method)

In [55]:
function possible_moves(possibility, i)
    (pos, space, cost) = possibility
    moves = []
    from = pos[i]
    for to in space
        if move_possible(possibility, from, to)
            push!(moves, to)
        end
    end
    return moves
end

possible_moves (generic function with 1 method)

In [65]:
possible_moves(possibilities[1],1), possible_moves(possibilities[1],2)

(Any[(1, 1), (2, 1), (4, 1), (6, 1), (8, 1), (10, 1), (11, 1)], Any[])

In [76]:
function way_length(from, to)
    if from[1] == to[1]
        return abs(from[2]-to[2])
    else
        return abs(from[1]-to[1]) + from[2]-1 + to[2]-1
    end
end

way_length (generic function with 1 method)

In [120]:
function one_step(possibilities, highscore)
    new_possibilities = []
    for possibility in possibilities
        (pos, space, cost) = possibility
        for i in 1:8
            moves = possible_moves(possibility,i)
            for move in moves
                new_cost = cost + way_length(pos[i],move) * step_cost[pos[i][3]]
                if new_cost < highscore
                    new_empty = setdiff(space, [move])
                    push!(new_empty, (pos[i][1],pos[i][2]))
                    new_pos = deepcopy(pos)
                    new_pos[i] = (move[1], move[2], new_pos[i][3])
                    push!(new_possibilities, (new_pos, new_empty, new_cost))
                end
            end
        end
    end
    return new_possibilities
end

one_step (generic function with 1 method)

In [121]:
one_step(possibilities, highscore)

28-element Vector{Any}:
 ([(8, 1, 'D'), (3, 3, 'B'), (5, 2, 'A'), (5, 3, 'A'), (7, 2, 'B'), (7, 3, 'D'), (9, 2, 'C'), (9, 3, 'C')], Set([(3, 2), (1, 1), (4, 1), (6, 1), (2, 1), (10, 1), (11, 1)]), 6000)
 ([(1, 1, 'D'), (3, 3, 'B'), (5, 2, 'A'), (5, 3, 'A'), (7, 2, 'B'), (7, 3, 'D'), (9, 2, 'C'), (9, 3, 'C')], Set([(8, 1), (3, 2), (4, 1), (6, 1), (2, 1), (10, 1), (11, 1)]), 3000)
 ([(4, 1, 'D'), (3, 3, 'B'), (5, 2, 'A'), (5, 3, 'A'), (7, 2, 'B'), (7, 3, 'D'), (9, 2, 'C'), (9, 3, 'C')], Set([(8, 1), (3, 2), (1, 1), (6, 1), (2, 1), (10, 1), (11, 1)]), 2000)
 ([(6, 1, 'D'), (3, 3, 'B'), (5, 2, 'A'), (5, 3, 'A'), (7, 2, 'B'), (7, 3, 'D'), (9, 2, 'C'), (9, 3, 'C')], Set([(8, 1), (3, 2), (1, 1), (4, 1), (2, 1), (10, 1), (11, 1)]), 4000)
 ([(2, 1, 'D'), (3, 3, 'B'), (5, 2, 'A'), (5, 3, 'A'), (7, 2, 'B'), (7, 3, 'D'), (9, 2, 'C'), (9, 3, 'C')], Set([(8, 1), (3, 2), (1, 1), (4, 1), (6, 1), (10, 1), (11, 1)]), 2000)
 ([(10, 1, 'D'), (3, 3, 'B'), (5, 2, 'A'), (5, 3, 'A'), (7, 2, 'B'), (7, 3, 'D'),

In [159]:
highscore = 20000
remaining_possibilities = possibilities
while length(remaining_possibilities) > 0
#for i in 1:20
    remaining_possibilities = one_step(remaining_possibilities, highscore)
    unique!(x->(Set(x[1]),x[3]),remaining_possibilities)
    finished_indices = is_finished.(remaining_possibilities)
    if any(finished_indices)
        highscore = minimum([x[3] for x in remaining_possibilities[finished_indices]])
        remaining_possibilities = remaining_possibilities[finished_indices .== 0]
    end
    println(length(remaining_possibilities))
end

28
353
2347
9200
23359
46717
86527
150765
246407
362057
451021
226035
117939
37905
4905
0


In [149]:
highscore

15516

## Part II

Add between 2 and 3:
```
  #D#C#B#A#
  #D#B#A#C#
```

In [160]:
empty_spaces = Set([(i,1) for i in [1,2,4,6,8,10,11]])
positions = [(3,2,'D'),(3,5,'B'),(5,2,'A'),(5,5,'A'),(7,2,'B'),(7,5,'D'),(9,2,'C'),(9,5,'C'),
             (3,3,'D'),(3,4,'D'),(5,3,'C'),(5,4,'B'),(7,3,'B'),(7,4,'A'),(9,3,'A'),(9,4,'C')]
possibilities = [(positions, empty_spaces, 0)]

1-element Vector{Tuple{Vector{Tuple{Int64, Int64, Char}}, Set{Tuple{Int64, Int64}}, Int64}}:
 ([(3, 2, 'D'), (3, 5, 'B'), (5, 2, 'A'), (5, 5, 'A'), (7, 2, 'B'), (7, 5, 'D'), (9, 2, 'C'), (9, 5, 'C'), (3, 3, 'D'), (3, 4, 'D'), (5, 3, 'C'), (5, 4, 'B'), (7, 3, 'B'), (7, 4, 'A'), (9, 3, 'A'), (9, 4, 'C')], Set([(8, 1), (1, 1), (4, 1), (6, 1), (2, 1), (10, 1), (11, 1)]), 0)

In [161]:
all_spaces = Set([empty_spaces..., [(x[1],x[2]) for x in positions]...])
occupied(empty) = setdiff(all_spaces, empty)

occupied (generic function with 1 method)

In [162]:
function way_blocked(from, to, space)
    occ = occupied(space)
    if from[2]>2 && any(x->x[1]==from[1] && x[2] ∈ 1:from[2]-1,occ)
        return true
    end
    if to[2]>2 && any(x->x[1]==to[1] && x[2] ∈ 1:to[2]-1,occ)
        return true
    end
    return any(x->x[1] ∈ min(from[1],to[1])+1:max(from[1],to[1])-1 && x[2]==1,occ)
end

way_blocked (generic function with 1 method)

In [163]:
function move_possible(possibility, from, to)
    (pos, space, cost) = possibility
    if from[2]==1 && to[2]==1
        return false
    end
    if way_blocked(from, to, space)
        return false
    end
    if from[1] == target_x[from[3]] && all(x->(from[1],x,from[3]) ∈ pos,from[2]:5)
        return false
    end
    if to[2]>1
        if to[1] != target_x[from[3]]
            return false
        end
        if to[2]<5 && any(x->(to[1],x,from[3]) ∉ pos,to[2]+1:5)
            return false
        end
    end
    return true
end

move_possible (generic function with 1 method)

In [164]:
function one_step(possibilities, highscore)
    new_possibilities = []
    for possibility in possibilities
        (pos, space, cost) = possibility
        for i in 1:16
            moves = possible_moves(possibility,i)
            for move in moves
                new_cost = cost + way_length(pos[i],move) * step_cost[pos[i][3]]
                if new_cost < highscore
                    new_empty = setdiff(space, [move])
                    push!(new_empty, (pos[i][1],pos[i][2]))
                    new_pos = deepcopy(pos)
                    new_pos[i] = (move[1], move[2], new_pos[i][3])
                    push!(new_possibilities, (new_pos, new_empty, new_cost))
                end
            end
        end
    end
    return new_possibilities
end

one_step (generic function with 1 method)

In [167]:
highscore = 50000
remaining_possibilities = possibilities
#while length(remaining_possibilities) > 0
for i in 1:30
    remaining_possibilities = one_step(remaining_possibilities, highscore)
    unique!(x->(Set(x[1]),x[3]),remaining_possibilities)
    finished_indices = is_finished.(remaining_possibilities)
    if any(finished_indices)
        highscore = minimum([x[3] for x in remaining_possibilities[finished_indices]])
        remaining_possibilities = remaining_possibilities[finished_indices .== 0]
    end
    println(length(remaining_possibilities))
end

28
353
2560
11300
29757
43855
33569
20373
27423
31665
30750
22978
14669
9400
9278
9186
9143
9028
9057
9817
11549
13500
15347
11538
11026
9754
7268
3773
1100
257


In [168]:
highscore

45272

Wow 😮‍💨